# Analisi preliminare con SQL

## Import, configurazione e caricamento database

In [1]:
import sqlite3
import pandas as pd
from IPython.display import display

db_path = '/kaggle/input/ecommerce-dataset/database_ecommerce.db' 

conn = sqlite3.connect(db_path)

def run_sql(query):
    try:
        df = pd.read_sql_query(query, conn)
        display(df)
    except Exception as e:
        print(f"Errore: {e}")

## Fatturato mensile netto

In [2]:
query = """
SELECT strftime("%Y-%m", o.order_date) as mese, 
       COUNT(DISTINCT o.order_id) as ordini_completati, 
       REPLACE(printf("€ %.2f", SUM(oi.quantity * oi.unit_price)), ".", ",") as fatturato_netto
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
WHERE o.status = "completed"
GROUP BY 1
ORDER BY 1;
"""
run_sql(query)

,mese,ordini_completati,fatturato_netto
0,2023-01,4,"€ 9630,79"
1,2023-02,9,"€ 21461,35"
2,2023-03,5,"€ 11919,77"
3,2023-04,2,"€ 6610,31"
4,2023-05,5,"€ 14990,08"
5,2023-06,6,"€ 13929,15"
6,2023-07,3,"€ 2911,69"
7,2023-08,5,"€ 7853,39"
8,2023-09,8,"€ 14537,61"
9,2023-10,6,"€ 12132,04"


Per costruire questa analisi temporale abbiamo dovuto lavorare sulla formattazione della data, poiché nel database è salvata in formato completo. Abbiamo utilizzato la funzione **strftime** per estrarre solo l'anno e il mese, trasformandoli nella chiave di raggruppamento principale della query. Per il calcolo economico, abbiamo moltiplicato la quantità per il prezzo unitario all'interno di una funzione di aggregazione **SUM**. Abbiamo utilizzato poi la clausola **WHERE** per filtrare esclusivamente gli ordini con stato "completed", escludendo a monte tutto ciò che non si è trasformato in incasso effettivo, per poi ordinare cronologicamente il risultato.

## Top 10 prodotti per fatturato

In [3]:
query = """
SELECT p.product_name, p.category, 
       SUM(oi.quantity) as quantita_venduta_netta, 
       REPLACE(printf("€ %.2f", SUM(oi.quantity * oi.unit_price)), ".", ",") as fatturato_netto
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
JOIN orders o ON oi.order_id = o.order_id
WHERE o.status = "completed"
GROUP BY 1, 2
ORDER BY SUM(oi.quantity * oi.unit_price) DESC
LIMIT 10;
"""
run_sql(query)

,product_name,category,quantita_venduta_netta,fatturato_netto
0,Product_5,Elettronica,30,"€ 14960,70"
1,Product_8,Sport,33,"€ 12550,89"
2,Product_13,Beauty,41,"€ 12391,84"
3,Product_12,Casa,35,"€ 12020,40"
4,Product_15,Beauty,34,"€ 10267,66"
5,Product_19,Casa,19,"€ 8964,01"
6,Product_20,Sport,24,"€ 8240,16"
7,Product_9,Elettronica,18,"€ 7774,92"
8,Product_3,Elettronica,23,"€ 6806,39"
9,Product_1,Casa,18,"€ 6768,72"


In questa query abbiamo dovuto unire informazioni provenienti da tre tabelle diverse tramite delle **JOIN**: ci servivano i dettagli dell'articolo da *order_items*, il nome da products e lo stato dell'ordine da orders. Abbiamo raggruppato i dati per nome prodotto e categoria, calcolando il fatturato totale come somma dei prezzi per le quantità. Abbiamo poi optato per un ordinamento decrescente (DESC) basato proprio su questa somma calcolata, combinato con il comando **LIMIT 10**, che ci ha permesso di limitare l'output e mostrare solo i prodotti che generano maggiore valore.

## Top 10 prodotti per quantità

In [4]:
query = """
SELECT p.product_name, p.category, SUM(oi.quantity) as quantita_netta
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
JOIN orders o ON oi.order_id = o.order_id
WHERE o.status = "completed"
GROUP BY 1, 2
ORDER BY 3 DESC
LIMIT 10;
"""
run_sql(query)

,product_name,category,quantita_netta
0,Product_11,Sport,51
1,Product_13,Beauty,41
2,Product_12,Casa,35
3,Product_15,Beauty,34
4,Product_8,Sport,33
5,Product_5,Elettronica,30
6,Product_10,Casa,27
7,Product_20,Sport,24
8,Product_3,Elettronica,23
9,Product_19,Casa,19


La struttura logica qui è speculare alla precedente, mantenendo le stesse **JOIN** tra le tre tabelle per garantire la coerenza dei dati. La differenza sostanziale risiede nella funzione di aggregazione scelta: invece di moltiplicare per il prezzo, abbiamo applicato la **SUM** direttamente sulla colonna delle quantità. Abbiamo modificato di conseguenza la clausola **ORDER BY** per farla puntare a questo nuovo valore aggregato.

## Fatturato per regione

In [5]:
query = """
SELECT c.region, COUNT(DISTINCT o.order_id) as ordini_netti, 
       REPLACE(printf("€ %.2f", SUM(oi.quantity * oi.unit_price)), ".", ",") as fatturato_regionale_netto
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
WHERE o.status = "completed"
GROUP BY 1
ORDER BY SUM(oi.quantity * oi.unit_price) DESC;
"""
run_sql(query)

,region,ordini_netti,fatturato_regionale_netto
0,Nord,29,"€ 54778,40"
1,Sud,24,"€ 50204,25"
2,Centro,10,"€ 26939,66"


Per questa analisi geografica abbiamo dovuto risalire la catena delle relazioni del database partendo dalla tabella customers, che contiene il dato sulla zona, fino ad arrivare a order_items per i valori economici. Abbiamo concatenato le tabelle usando le chiavi esterne *customer_id* e *order_id*. Una volta collegate le tabelle, abbiamo raggruppato tutto per la colonna *region*. Anche qui, il filtro sullo stato "completed" è stato essenziale per attribuire alle regioni solo il fatturato effettivamente consolidato, evitando di contare ordini che magari sono stati annullati o resi.

## Performance per categoria

In [6]:
query = """
SELECT p.category, COUNT(DISTINCT o.order_id) as ordini_venduti, 
       REPLACE(printf("€ %.2f", SUM(oi.quantity * oi.unit_price)), ".", ",") as fatturato_totale
FROM products p
JOIN order_items oi ON p.product_id = oi.product_id
JOIN orders o ON oi.order_id = o.order_id
WHERE o.status = "completed"
GROUP BY 1
ORDER BY SUM(oi.quantity * oi.unit_price) DESC;
"""
run_sql(query)

,category,ordini_venduti,fatturato_totale
0,Casa,37,"€ 41515,62"
1,Sport,37,"€ 31900,14"
2,Elettronica,19,"€ 29542,01"
3,Beauty,25,"€ 28964,54"


Qui abbiamo voluto aggregare i dati a un livello superiore rispetto al singolo prodotto. Abbiamo sfruttato la tabella products come punto di partenza per estrarre la categoria e l'abbiamo messa in relazione con gli articoli venduti. Il raggruppamento tramite **GROUP BY** category ci ha permesso di condensare le tante righe di vendita in poche righe riassuntive. Abbiamo scelto di contare gli ordini distinti (**COUNT DISTINCT**) per capire la frequenza di acquisto e contemporaneamente sommare il valore economico, così da avere in un'unica vista sia il volume di traffico che il valore monetario di ogni settore merceologico.

## Clienti nuovi vs ricorrenti

In [7]:
query = """
SELECT tipo_cliente, COUNT(customer_id) as numero_clienti, 
       REPLACE(printf("€ %.2f", SUM(spesa_totale)), ".", ",") as fatturato_generato
FROM (
    SELECT c.customer_id, SUM(oi.quantity * oi.unit_price) as spesa_totale,
    CASE WHEN COUNT(DISTINCT o.order_id) > 2 THEN "Ricorrente" ELSE "Nuovo" END as tipo_cliente
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    WHERE o.status = "completed"
    GROUP BY 1
)
GROUP BY 1;
"""
run_sql(query)

,tipo_cliente,numero_clienti,fatturato_generato
0,Nuovo,29,"€ 85656,43"
1,Ricorrente,8,"€ 46265,88"


Abbiamo utilizzato una subquery per creare una tabella temporanea virtuale in cui, per ogni cliente, abbiamo calcolato il numero di ordini distinti effettuati. All'interno di questa subquery abbiamo implementato una logica condizionale con **CASE WHEN**: se il conteggio degli ordini era superiore a due, abbiamo etichettato il cliente come "Ricorrente", altrimenti come "Nuovo". La query esterna ha poi preso questi risultati e li ha raggruppati per l'etichetta appena creata, permettendoci di sommare il fatturato generato da questi due segmenti comportamentali.

## Valore ordini resi e annullati

In [8]:
query = """
SELECT o.status as esito, COUNT(DISTINCT o.order_id) as numero_ordini, 
       REPLACE(printf("€ %.2f", SUM(oi.quantity * oi.unit_price)), ".", ",") as valore_rimborsato
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
WHERE o.status IN ("cancelled", "returned")
GROUP BY 1
ORDER BY SUM(oi.quantity * oi.unit_price) DESC;
"""
run_sql(query)

,esito,numero_ordini,valore_rimborsato
0,returned,71,"€ 166461,55"
1,cancelled,66,"€ 147108,38"


In questo caso abbiamo utilizzato la clausola **WHERE** con l'operatore **IN** per intercettare due stati diversi contemporaneamente: "cancelled" e "returned". Invece di escluderli come fatto nelle altre query, qui li abbiamo isolati. Raggruppando per lo stato dell'ordine, abbiamo potuto sommare il valore dei prodotti contenuti in queste transazioni. È stato importante usare la tabella *order_items* anche in questo contesto perché il valore del reso dipende dalla quantità specifica e dal prezzo storico dell'articolo al momento dell'acquisto, dato che abbiamo recuperato proprio da lì.

## Scontrino medio

In [9]:
query = """
SELECT 
    REPLACE(printf("€ %.2f", AVG(totale_ordine)), ".", ",") as scontrino_medio
FROM (
    SELECT o.order_id, SUM(oi.quantity * oi.unit_price) as totale_ordine
    FROM orders o
    JOIN order_items oi ON o.order_id = oi.order_id
    WHERE o.status = "completed"
    GROUP BY o.order_id
);
"""
run_sql(query)

,scontrino_medio
0,"€ 2094,00"


Per calcolare correttamente lo scontrino medio non potevamo fare una semplice media sulle righe dei prodotti, perché avrebbe falsato il risultato. Abbiamo dovuto strutturare la query su due livelli: nella subquery interna abbiamo calcolato il valore totale di ogni singolo ordine sommando i prezzi dei suoi articoli e raggruppando per *order_id*. Solo successivamente, nella query esterna, abbiamo applicato la funzione di aggregazione AVG su questi totali già calcolati. Abbiamo ritenuto questo doppio passaggio l'unico modo matematicamente corretto per ottenere la media del valore dei carrelli e non la media del prezzo dei prodotti.

## Categorie con più resi

In [10]:
query = """
SELECT p.category, SUM(oi.quantity) as quantita_articoli_resi, 
       REPLACE(printf("€ %.2f", SUM(oi.quantity * oi.unit_price)), ".", ",") as valore_perso
FROM returns r
JOIN order_items oi ON r.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
GROUP BY 1
ORDER BY 2 DESC;
"""
run_sql(query)

,category,quantita_articoli_resi,valore_perso
0,Casa,237,"€ 69690,16"
1,Sport,128,"€ 28897,54"
2,Elettronica,116,"€ 41580,57"
3,Beauty,101,"€ 26293,28"


Per analizzare i resi abbiamo dovuto introdurre nel flusso la tabella returns. Abbiamo eseguito una **JOIN** tra questa tabella, gli articoli dell'ordine e i prodotti per risalire alla categoria di appartenenza della merce restituita. Abbiamo scelto di raggruppare per categoria e di ordinare i risultati in base alla somma delle quantità restituite. Questo approccio ci ha permesso di vedere non solo il valore economico perso, ma soprattutto quali tipologie di prodotti sono più soggette a insoddisfazione da parte del cliente.

## Analisi metodi di pagamento

In [11]:
query = """
SELECT 
    pay.payment_method as metodo,
    COUNT(DISTINCT o.order_id) as numero_transazioni,
    REPLACE(printf("€ %.2f", SUM(pay.amount)), ".", ",") as volume_transatazioni
FROM payments pay
JOIN orders o ON pay.order_id = o.order_id
WHERE o.status = "completed"
GROUP BY 1
ORDER BY 3 DESC;
"""
run_sql(query)

,metodo,numero_transazioni,volume_transatazioni
0,Bonifico,23,"€ 16555,51"
1,Carta,20,"€ 11138,11"
2,PayPal,20,"€ 10947,25"


L'ultima analisi si basa sulla tabella *payments*. Abbiamo collegato questa tabella agli ordini per poter filtrare, ancora una volta, solo quelli con stato "completed". Abbiamo raggruppato i dati per la colonna payment_method e abbiamo calcolato due metriche distinte: il conteggio delle transazioni uniche per vedere il metodo più usato e la somma degli importi per vedere quello che muove più denaro.

## Chiusura connessione

In [12]:
conn.close()